In [9]:
import duckdb
import pandas as pd

conn = duckdb.connect()

In [2]:
books_ids = pd.read_csv('data/googlereads/book_id_map.csv')

In [3]:
books_ids

,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361
...,...,...
2360645,2360645,19517100
2360646,2360646,18597299
2360647,2360647,18584882
2360648,2360648,18518801


In [4]:
users_ids = pd.read_csv('data/googlereads/user_id_map.csv')

In [5]:
users_ids

,user_id_csv,user_id
0,0,8842281e1d1347389f2ab93d60773d4d
1,1,72fb0d0087d28c832f15776b0d936598
2,2,ab2923b738ea3082f5f3efcbbfacb218
3,3,d986f354a045ffb91234e4af4d1b12fd
4,4,7504b2aee1ecb5b2872d3da381c6c91e
...,...,...
876140,876140,d890e8079d8896e0cc6c4f178634850a
876141,876141,71ca1aedba3bb07e9e914d072add7e48
876142,876142,d0f6d1a4edcab80a6010cfcfeda4999f
876143,876143,b0883ebf8e31731f1c5d91e678c26666


In [6]:
interactions = pd.read_csv(
    filepath_or_buffer='data/googlereads/goodreads_interactions.csv',
    dtype={
        'is_read': pd.BooleanDtype,
        'is_read': pd.UInt8Dtype,
        'is_reviewed': pd.BooleanDtype
    },
)

In [7]:
interactions

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0
...,...,...,...,...,...
228648337,876144,24772,0,0,0
228648338,876144,23847,1,4,0
228648339,876144,23950,1,3,0
228648340,876144,374106,1,5,1


In [15]:
df_google_reads = conn.execute("""
SELECT
  u.user_id,
  b.book_id,
  i.rating
FROM interactions i
    LEFT JOIN users_ids u ON i.user_id = u.user_id_csv
    LEFT JOIN books_ids b ON i.book_id = b.book_id_csv
WHERE
  i.is_read = 1 AND i.is_reviewed = 1
"""
).fetchdf()

In [16]:
df_google_reads

,user_id,book_id,rating
0,8842281e1d1347389f2ab93d60773d4d,21,5
1,8842281e1d1347389f2ab93d60773d4d,2,5
2,8842281e1d1347389f2ab93d60773d4d,6,5
3,8842281e1d1347389f2ab93d60773d4d,1022863,5
4,8842281e1d1347389f2ab93d60773d4d,828,2
...,...,...,...
15935232,fb0401717400256f6ef0c90e8d91df44,43884,5
15935233,fb0401717400256f6ef0c90e8d91df44,16130549,5
15935234,e518c4bc48b853f0fa811c519263f46d,149267,0
15935235,791c0c267bb4da2ce63fc1b05a91a2af,1081701,5


In [25]:
df_google_reads.to_parquet(
    path='./data/googlereads/refined/google_reads.parquet.snappy',
    compression='snappy',
    index=False,
)

In [21]:
user_group = conn.execute("""
SELECT
  user_id,
  COUNT(*) AS number_of_reviews
FROM df_google_reads
GROUP BY user_id
HAVING COUNT(*) > 10
ORDER BY 2 DESC
"""
).fetchdf()

In [22]:
user_group

,user_id,number_of_reviews
0,459a6c4decf925aedd08e45045c0d8c6,9542
1,9003d274774f4c47e62f77600b08ac1d,7767
2,843a44e2499ba9362b47a089b0b0ce75,7571
3,b7772313835ce6257a3fbe7ad2649a29,7130
4,dd9785b14664103617304996541ed77a,6379
...,...,...
183359,6a517eebbd91daa8737d006e15a4a73f,11
183360,51665b2670d8cbd4f22a22824083c559,11
183361,329ff62ae98ed39a9006ec81aae6f9df,11
183362,831f476870155162722880d99bca0fe2,11
